<a href="https://colab.research.google.com/github/Schify/FacExcerciseBook/blob/main/Downloading_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#with this piece of code you can connect your drive with the google collab
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#In the github repository we will provide the google drive link to the zip file
#containing the data that we will use. 
#This will unzip it into a train and a val folder
!unzip "gdrive/MyDrive/DL_big_homework/fairface"

A streamkimeneten csak az utolsó 5000 sor látható.
  inflating: val/5370.jpg            
  inflating: val/5596.jpg            
  inflating: val/5875.jpg            
  inflating: val/5878.jpg            
  inflating: val/5879.jpg            
  inflating: val/5883.jpg            
  inflating: val/6126.jpg            
  inflating: val/6132.jpg            
  inflating: val/6144.jpg            
  inflating: val/6150.jpg            
  inflating: val/6340.jpg            
  inflating: val/7190.jpg            
  inflating: val/7734.jpg            
  inflating: val/7747.jpg            
  inflating: val/7872.jpg            
  inflating: val/7940.jpg            
  inflating: val/8065.jpg            
  inflating: val/8069.jpg            
  inflating: val/8125.jpg            
  inflating: val/8202.jpg            
  inflating: val/8204.jpg            
  inflating: val/8205.jpg            
  inflating: val/8268.jpg            
  inflating: val/8270.jpg            
  inflating: val/8281.jpg            

In [3]:
#loading all the necessary libraries for the project
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import cv2
from tqdm.notebook import tqdm #useful module for progress bars
import logging
import gc
from datetime import datetime
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.image import ImageDataGenerator
from skimage import io
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm as tqdm
from datetime import datetime
!pip install keras_facenet
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.models import load_model
from keras_facenet import FaceNet
import shutil
embedder = FaceNet()

     |████████████████████████████████| 2.3 MB 35.3 MB/s 
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10386 sha256=daf61402ac769f761cafec282a0e300976a18eef517ff67879a57d0a0f6c4211
  Stored in directory: /root/.cache/pip/wheels/2b/76/1a/a29ceb07bfca098e2b0bc8690d550753a0384cbed26d5f47ee
Successfully built keras-facenet


In [4]:
#here this is a function for loading the images from a given directory.
def load_all_images(directory, img_size = False, range_from = 0, range_to = 1):
    filenames = os.listdir(directory)
    num = len(filenames)
    resize = True
    if(not img_size):
        img = cv2.imread(os.path.join(directory,filenames[0]))
        img_size = img.shape[0]
        resize = False
    #For memory sizes this big dynamic arrays (lists) are not an effective way to reserve memory
    #Hence numpy arrays are utalized
    print(f"{int(num*(range_to-range_from))} files to be loaded")
    start = int(num*range_from)
    images = np.zeros(shape=(int(num*(range_to-range_from)), img_size, img_size, 3), dtype=np.float16)
    w = 100
    num_none = 0
    for i in tqdm(range(int(images.shape[0]))):
        img = cv2.imread(os.path.join(directory,filenames[i+start]))
        if img is not None:

            if(resize):
                images[i, ...] = cv2.resize(img, (img_size, img_size))
            else:
                images[i, ...] = img
            '''
            w-=1
            if(__debug__ and w==0):
                break
        else:
            num_none +=1
'''
    if(num_none>0):
        print(f"{num_none} images were unable to load, or not valid image types")
    return images

def load_all_images_embedded(directory, range_from = 0, range_to = 1):
    filenames = os.listdir(directory)
    num = len(filenames)
    #For memory sizes this big dynamic arrays (lists) are not an effective way to reserve memory
    #Hence numpy arrays are utalized
    print(f"{int(num*(range_to-range_from))} files to be loaded")
    start = int(num*range_from)
    image_embeddings = np.zeros(shape=(int(num*(range_to-range_from)), 512), dtype=np.float16)
    num_none = 0
    for i in tqdm(range(int(image_embeddings.shape[0]))):
        img_embedding = embedder.extract(directory+"/"+filenames[i+start], threshold=0.95)
        if len(img_embedding)>0:
            image_embeddings[i, :] = img_embedding[0]["embedding"]
        else:
          num_none += 1
    if(num_none>0):
        print(f"{num_none} images were unable to load, or not valid image types")
    return image_embeddings

In [ ]:
#Here we used the garbage collector library to collect the pictures into a npz dataformat.
#we are splitting it into multiple batches and making the images 128x128 sizes.
#This way we are reserving memory and it's easier to load this later.
gc.collect()
np.savez("train_128x128_1_of_4.npz", train=load_all_images("train",
img_size=128,
range_from=0,
 range_to=0.25)
 )
gc.collect()
np.savez("train_128x128_2_of_4.npz", train=load_all_images("train",
img_size=128,
range_from=0.25,
 range_to=0.5)
 )
gc.collect()
np.savez("train_128x128_3_of_4.npz", train=load_all_images("train",
img_size=128,
range_from=0.5,
 range_to=0.75)
 )
gc.collect()
np.savez("train_128x128_4_of_4.npz", train=load_all_images("train",
img_size=128,
range_from=0.75,
 range_to=0.95)
 )

gc.collect()
np.savez("test_128x128.npz", train=load_all_images("train",
img_size=128,
range_from=0.95,
 range_to=1)
 )
gc.collect()
np.savez("val_128x128.npz", val=load_all_images("val",
img_size=128)
 )
gc.collect()

21686 files to be loaded


  0%|          | 0/21686 [00:00<?, ?it/s]

21686 files to be loaded


  0%|          | 0/21686 [00:00<?, ?it/s]

21686 files to be loaded


...

4337 files to be loaded


  0%|          | 0/4337 [00:00<?, ?it/s]

10954 files to be loaded


  0%|          | 0/10954 [00:00<?, ?it/s]

18

In [ ]:
#moving the files into our drives so we don't have to create the files every time
#we can just load them through the drive


In [6]:
#Here we load and embed the images. This takes a few minutes.
np.savez("train_embedding.npz", train=load_all_images_embedded("train", range_to = 0.06))
gc.collect()

5204 files to be loaded


  0%|          | 0/5204 [00:00<?, ?it/s]

457 images were unable to load, or not valid image types


21240

In [7]:
#Here we load and embed the images. This takes a few minutes.
np.savez("val_embedding.npz", val=load_all_images_embedded("val", range_to = 0.05))
gc.collect()

547 files to be loaded


  0%|          | 0/547 [00:00<?, ?it/s]

41 images were unable to load, or not valid image types


44813

In [8]:
np.savez("test_embedding.npz", val=load_all_images_embedded("val", range_to = 0.005))
gc.collect()

54 files to be loaded


  0%|          | 0/54 [00:00<?, ?it/s]

2 images were unable to load, or not valid image types


59761

In [10]:
#Here we move the created files into our drive so we don't have to do this every time we start the colab and work on the project.
files = ["test_embedding.npz",'val_embedding.npz', "train_embedding.npz"]
for f in files:
    shutil.move(f, 'gdrive/MyDrive/DL_big_homework/')